In [96]:
# import argparse
import os
import pickle
# import time
# from itertools import product
# from math import inf, pi

import jax
import jax.numpy as jnp
from jax.experimental.ode import odeint
import jax.debug as jdb
from functools import partial
import numpy as np

from tqdm.auto import tqdm

# from dynamics import prior, disturbance, plant
# from utils import params_to_posdef
from utils import random_ragged_spline, spline
from utils import (tree_normsq, rk38_step, epoch,   # noqa: E402
                   odeint_fixed_step, odeint_ckpt, random_ragged_spline, spline,
            params_to_cholesky, params_to_posdef, vee, hat,
            
            quaternion_to_rotation_matrix, flat_rotation_matrix_to_quaternion, 
            quaternion_multiply)

# from functools import partial
import matplotlib.pyplot as plt
# import csv

plt.rcParams.update({'font.size': 24})

In [132]:
# trial_name = 'hardware_reg_P_1e-1'
# pkl = 'seed=0_M=50_E=1000_pinit=2.00_pfreq=2000_regP=0.1000.pkl'
trial_name = 'hardware_07'
pkl = 'seed=0_M=50_E=1000_pinit=2.00_pfreq=2000_regP=0.0020.pkl'
filename = f'{trial_name}/{pkl}'
with open(f'train_results/{filename}', 'rb') as file:
    raw = pickle.load(file)

In [133]:
epoch_i = 800
# mystery_i = 0
# for k,v in raw['train_lossaux_history'][epoch_i].items():
#     if type(v) is list or type(v) is tuple:
#         for i in v:
#             print(k, i.shape)
#     else:
#         print(k, v.shape)

def validate_model(raw, epoch_i):
    # hparams = raw['hparams']
    # dt = hparams['meta']['dt']
    # T = hparams['meta']['T']
    # coefs = raw['coefs']
    # max_ref = raw['max_ref']
    # min_ref = raw['min_ref']
    # t_knots = raw['t_knots']

    # A = raw['train_lossaux_history'][epoch_i]['A']
    # W = raw['train_lossaux_history'][epoch_i]['W']
    # b = raw['train_lossaux_history'][epoch_i]['b']
    # control_loss = raw['train_lossaux_history'][epoch_i]['control_loss']
    # eigs_K = raw['train_lossaux_history'][epoch_i]['eigs_K']
    eigs_P = raw['train_lossaux_history'][epoch_i]['eigs_P']
    eigs_Λ = raw['train_lossaux_history'][epoch_i]['eigs_Λ']
    estimation_loss = raw['train_lossaux_history'][epoch_i]['estimation_loss']
    l2_penalty = raw['train_lossaux_history'][epoch_i]['l2_penalty']
    pnorm = raw['train_lossaux_history'][epoch_i]['pnorm']
    reg_P_penalty = raw['train_lossaux_history'][epoch_i]['reg_P_penalty']
    tracking_loss = raw['train_lossaux_history'][epoch_i]['tracking_loss']
    # x = raw['train_lossaux_history'][epoch_i]['x']

    print('eigs_P:', eigs_P)


    # print(raw['best_step_meta'])

    # print(x.shape)
    # print(t_knots.shape)
    # print(coefs[0].shape)
    print('A', A.shape)

    mystery_i = 0
    
    def reference(t):
        # print([spline(t, t_knots[mystery_i], c[mystery_i]) for c in coefs])
        r = jnp.array([spline(t, t_knots[mystery_i], c[mystery_i]) for c in coefs]) + jnp.array([0, 0, 1])
        r = jnp.clip(r, min_ref, max_ref)
        return r

    num_steps = int(np.maximum(np.abs((T - 0)/dt), 1))
    ts = jnp.linspace(0, T, num_steps + 1)
    r = jax.vmap(reference)(ts)
    dr = jax.vmap(jax.jacfwd(reference))(ts)

    fig, ax = plt.subplots(1, 3, figsize=(30, 10))
    ax[0].plot(ts, x[:, 0], 'r-', label='q')
    ax[0].plot(ts, r[:, 0], 'b--', label='r')
    ax[0].set_ylabel('x')
    ax[0].set_xlabel('t')
    ax[0].legend()

    ax[1].plot(ts, x[:, 1], 'r-', label='q')
    ax[1].plot(ts, r[:, 1], 'b--', label='r')
    ax[1].set_ylabel('y')
    ax[1].set_xlabel('t')
    ax[1].legend()

    ax[2].plot(ts, x[:, 2], 'r-', label='q')
    ax[2].plot(ts, r[:, 2], 'b--', label='r')
    ax[2].set_ylabel('z')
    ax[2].set_xlabel('t')
    ax[2].legend()
    
    fig.suptitle(f'Position')
    fig.tight_layout()

    fig, ax = plt.subplots(1, 3, figsize=(30, 10))
    ax[0].plot(ts, x[:, 3], 'r-', label='dq')
    ax[0].plot(ts, dr[:, 0], 'b--', label='dr')
    ax[0].set_ylabel('dx')
    ax[0].set_xlabel('t')
    ax[0].legend()

    ax[1].plot(ts, x[:, 4], 'r-', label='dq')
    ax[1].plot(ts, dr[:, 1], 'b--', label='dr')
    ax[1].set_ylabel('dy')
    ax[1].set_xlabel('t')
    ax[1].legend()

    ax[2].plot(ts, x[:, 5], 'r-', label='dq')
    ax[2].plot(ts, dr[:, 2], 'b--', label='dr')
    ax[2].set_ylabel('dz')
    ax[2].set_xlabel('t')
    ax[2].legend()
    
    fig.suptitle(f'Velocity')
    fig.tight_layout()

    A_norm = jnp.linalg.norm(A, axis=(1,2))

    fig, ax = plt.subplots(1, 2, figsize=(30, 10))

    ax[0].plot(ts, A_norm, label='A')
    ax[0].set_ylabel('A')
    ax[0].set_xlabel('t')
    ax[0].legend()

    print('eigs_P:', eigs_P)

    P_small = 1e-3*jnp.eye(32)

    print('eigs_P_small:', jnp.linalg.eigvals(P_small))

validate_model(raw, epoch_i)

# print(len(raw['train_lossaux_history']))

eigs_P: [1.417242   1.1743294  0.9401242  0.8727     0.7981379  0.69925547
 0.7417965  0.7625276  1.0175765  0.9254214  1.3088002  0.93883204
 0.96673393 0.9502355  0.7714687  0.8862527  1.1462444  1.0863158
 1.2435273  1.0232772  1.3346109  1.1533848  1.4100971  0.89633495
 1.5871848  0.6142442  0.755523   0.8840394  0.75985986 1.3065823
 0.95584226 1.2999767 ]


NameError: name 'A' is not defined

In [95]:
epoch_i = 999
# mystery_i = 0
# for k,v in raw['train_lossaux_history'][epoch_i].items():
#     if type(v) is list or type(v) is tuple:
#         for i in v:
#             print(k, i.shape)
#     else:
#         print(k, v.shape)

def validate_model(raw, epoch_i):
    hparams = raw['hparams']
    dt = hparams['meta']['dt']
    T = hparams['meta']['T']

    for epoch in range(2): 

        A = raw['train_lossaux_history'][epoch_i]['A']
        Omega = raw['train_lossaux_history'][epoch_i]['Omega']
        R_flatten = raw['train_lossaux_history'][epoch_i]['R_flatten']
        W = raw['train_lossaux_history'][epoch_i]['W']
        b = raw['train_lossaux_history'][epoch_i]['b']
        coefs = raw['train_lossaux_history'][epoch_i]['coefs']
        control_loss = raw['train_lossaux_history'][epoch_i]['control_loss']
        eigs_K = raw['train_lossaux_history'][epoch_i]['eigs_K']
        eigs_P = raw['train_lossaux_history'][epoch_i]['eigs_P']
        eigs_Λ = raw['train_lossaux_history'][epoch_i]['eigs_Λ']
        estimation_loss = raw['train_lossaux_history'][epoch_i]['estimation_loss']
        l2_penalty = raw['train_lossaux_history'][epoch_i]['l2_penalty']
        max_ref = raw['train_lossaux_history'][epoch_i]['max_ref']
        min_ref = raw['train_lossaux_history'][epoch_i]['min_ref']
        pnorm = raw['train_lossaux_history'][epoch_i]['pnorm']
        reg_P_penalty = raw['train_lossaux_history'][epoch_i]['reg_P_penalty']
        t_knots = raw['train_lossaux_history'][epoch_i]['t_knots']
        tracking_loss = raw['train_lossaux_history'][epoch_i]['tracking_loss']
        x = raw['train_lossaux_history'][epoch_i]['x']

        print('t_knots', t_knots[0])
        for c in coefs:
            print('c', c[0])
        print('min_ref', min_ref)
        print('max_ref', max_ref)  

        


validate_model(raw, epoch_i)

# print(len(raw['train_lossaux_history']))

t_knots [0.        1.4670889 2.8217154 3.3582501 4.407982  5.       ]
c [[ 0.0000000e+00  9.5485004e-05  0.0000000e+00  0.0000000e+00
  -2.7279516e+01  5.0996490e+01 -3.1402275e+01  6.0300245e+00
   3.7205747e-01 -8.1280664e-02]
 [-1.3644080e+00  1.7599494e+00  7.9936833e+00 -1.6846166e+00
  -1.1977000e+01  1.6155930e-01  8.9312115e+00 -3.0787194e+00
  -1.9575220e-01  4.2669162e-02]
 [ 5.8857727e-01 -1.4070526e+00 -3.0378637e-01  9.6397012e-01
   1.6531733e-01 -1.6414899e-01 -5.1564630e-02  1.8631341e-02
   3.9791032e-03 -9.3978568e-04]
 [-1.8701315e-01  1.0999930e+00  6.2544460e+00 -1.8840013e+00
  -9.0360432e+00  8.9352489e-01  6.4360371e+00 -2.3527346e+00
  -1.8631709e-01  4.0996507e-02]
 [ 1.0788884e+00 -1.5194353e+00 -4.0916395e-01  1.8461752e+00
   1.7701709e-01 -8.8104045e-01 -3.6802265e-01  3.7562007e-01
   1.2460595e-02 -2.7129450e-03]]
c [[ 0.0000000e+00 -2.0861324e-05  0.0000000e+00  0.0000000e+00
   1.3989177e+01 -2.2290104e+01  1.2224873e+01 -2.1511908e+00
  -1.3490152e-01